In [1]:
import datasets

/data/kcl/lpy/anaconda3/envs/verl_new/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/kcl/lpy/anaconda3/envs/verl_new/lib/python3.12/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [15]:
data_path = '/data/kcl/lpy/data/OpenThoughts'
cache_path = '/data/kcl/lpy/hf'
dataset = datasets.load_dataset(data_path, "metadata", cache_dir=cache_path, split="train")

Generating train split: 100%|██████████| 113957/113957 [00:12<00:00, 9465.64 examples/s]


In [33]:
    data_source = "open-thoughts/OpenThoughts-114k"
    instruction_following = 'Let\'s think step by step and output the final answer after "####".'
    def make_map_fn(split):
        def process_fn(example, idx):
            question_raw = example.pop("problem")

            question = question_raw + " " + instruction_following

            answer_raw = example.pop("ground_truth_solution")
            solution = answer_raw
            data = {
                "data_source": data_source,
                "prompt": [
                    {
                        "role": "user",
                        "content": question,
                    }
                ],
                "ability": "math",
                "reward_model": {"style": "rule", "ground_truth": solution},
                "extra_info": {
                    "split": 'train',
                    "index": idx,
                    "answer": answer_raw,
                    "question": question_raw,
                },
            }
            return data

        return process_fn

    dataset = dataset.map(function=make_map_fn('train'), with_indices=True)

Map: 100%|██████████| 113957/113957 [00:21<00:00, 5374.89 examples/s]


In [36]:
local_dir = '/data/kcl/lpy/data/OpenThoughts'
import os
dataset.to_parquet(os.path.join(local_dir, "train_verl.parquet"))

Creating parquet from Arrow format: 100%|██████████| 53/53 [00:27<00:00,  1.95ba/s]


5241699449

In [42]:
print(dataset['reward_model'][0]['ground_truth'])

1. Consider the given inequality:

\[
\frac{1}{a(1+b)}+ \frac{1}{b(1+c)}+ \frac{1}{c(1+a)} \geq \frac{3}{1 + abc}
\]

To simplify, we add \( \frac{3}{1 + abc} \) to both sides. The new inequality becomes:

\[
\frac{1}{a(1+b)} + \frac{1}{b(1+c)} + \frac{1}{c(1+a)} + \frac{3}{1 + abc} \geq \frac{6}{1 + abc}
\]

2. Let's analyze each term with an added \( \frac{1}{1 + abc} \):

\[
\frac{1}{a(1+b)} + \frac{1}{1 + abc}, \quad \frac{1}{b(1+c)} + \frac{1}{1 + abc}, \quad \frac{1}{c(1+a)} + \frac{1}{1 + abc}
\]

We can rewrite them as follows:

\[
\begin{aligned}
\frac{1}{a(1+b)} + \frac{1}{1 + abc} &= \frac{1}{1 + abc} \left( \frac{1 + abc}{a(1+b)} + 1 \right), \\
\frac{1}{b(1+c)} + \frac{1}{1 + abc} &= \frac{1}{1 + abc} \left( \frac{1 + abc}{b(1+c)} + 1 \right), \\
\frac{1}{c(1+a)} + \frac{1}{1 + abc} &= \frac{1}{1 + abc} \left( \frac{1 + abc}{c(1+a)} + 1 \right).
\end{aligned}
\]

3. Simplifying inside the parentheses:

\[
\begin{aligned}
\frac{1 + abc}{a(1+b)} + 1 &= \frac{1 + abc + a(1+b)